In [2]:
import pandas as pd
import requests
import yfinance as yf
import warnings
warnings.simplefilter(action='igre', category=FutureWarning)

/var/folders/pf/fm0v0g_52r754cfv3r4lqv980000gn/T/ipykernel_75523/3286336062.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [67]:
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}

In [101]:
def idv_stock_rec(stock_symbol: str) -> dict:
    """
    Get stock data from Alpha Vantage API
    request arguments: the symbol of the stock
    returns: the in depth information of the stock including:
        name and other basic stock information like data,
        strongestPrinciple and source/reasoning for strongest principle as a short blurb,
        weakestPrinciple and source/reasoning for weakest principle as a short blurb,
        relevant news articles, 
    """

    headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}

    stock = yf.Ticker(stock_symbol)
    shortened_info = {}
    for key in ['symbol', 'shortName', 'longName','longBusinessSummary', 'totalRevenue', 'beta', 'overallRisk']:
        try:
            shortened_info['key'] = stock.info[key]
        except:
            shortened_info['key'] = 'N/A'
    info = shortened_info
    
    hist = stock.history(period="1mo")['Close']
    hist = hist.reset_index()
    hist = hist[['Close']].to_dict()
    info['price_history'] = hist
    
    url = "https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={}".format(stock_symbol)
    response = requests.get(url, headers=headers)
    try:
        esg_data = response.json()
    except:
        esg_data = {'esgChart': {'result': [{}]}}
    
    if esg_data['esgChart']['result'] == [{}]:
        esg_score = 'N/A'
        env_score = 'N/A'
        gov_score = 'N/A'
        soc_score = 'N/A'
    else:
        esg_score = esg_data['esgChart']['result'][0]['symbolSeries']['esgScore'][0]
        env_score = esg_data['esgChart']['result'][0]['symbolSeries']['environmentScore'][0]
        gov_score = esg_data['esgChart']['result'][0]['symbolSeries']['governanceScore'][0]
        soc_score = esg_data['esgChart']['result'][0]['symbolSeries']['socialScore'][0]

    scores = {env_score: 'environmentScore', gov_score: 'governanceScore', soc_score: 'socialScore'}
    min_score = scores[min(scores.keys())]
    max_score = scores[max(scores.keys())]

    url = 'https://ca.finance.yahoo.com/quote/{}/sustainability'.format(stock_symbol)
    try:
        involvements = pd.read_html(url, storage_options=headers, header=None)
        involvements = involvements[1]
        for i in [0, 1, 2, 3]:
            if involvements.iloc[i][0] == 'Yes':
                info.iloc['vice_products'] = 'Yes'
            if i == 3:
                if involvements.iloc[i][0] == 'No':
                    info.iloc['vice_products'] = 'No'
        for i in [4, 5, 13]:
            if involvements.iloc[i][0] == 'Yes':
                info.iloc['ethical_concerns'] = 'Yes'
            if i == 12:
                if involvements[i][0] == 'No':
                    info.iloc['ethical_concerns'] = 'No'
        for i in [6, 7, 10]:
            if involvements.iloc[i][0] == 'Yes':
                info.iloc['military_involvement'] = 'Yes'
            if i == 10:
                if involvements.iloc[i][0] == 'No':
                    info.iloc['military_involvement'] = 'No'
        for i  in [9, 11, 12]:
            if involvements.iloc[i][0] == 'Yes':
                info.iloc['health_impact'] = 'Yes'
            if i == 12:
                if involvements.iloc[i][0] == 'No':
                    info.iloc['health_impact'] = 'No'
        if involvements.iloc[8][0] == 'Yes':
            info['catholic_values'] = 'Yes'
        else:
            info['catholic_values'] = 'No'
    except:
        info['vice_products'] = 'N/A'
        info['ethical_concerns'] = 'N/A'
        info['military_involvement'] = 'N/A'
        info['catholic_values'] = 'N/A'
        info['health_impact'] = 'N/A'
    
    info['esg_score'] = esg_score
    info['env_score'] = env_score
    info['gov_score'] = gov_score
    info['soc_score'] = soc_score
    info['strongest_principle'] = max_score
    info['weakest_principle'] = min_score

    return info

In [99]:
def stock_rec(selected_filters: list) -> dict:
    """
    Get stock recommendation from Sovestico API
    //request arguments: a list of principles to filter by
    // returns: paginated data of top suggestions
    {
        suggestions:
            [
                {
                    symbol: "",
                    name: "",
                    price: 0.0,
                    change: -0.07,
                    strongestPrinciple: "",
                    weakestPrinciple: "",
                    esg: 0,
                    stockData: [//dependent on what API we decide to use]
                },
                ...
            ]
    }
    """
    tickers = pd.read_csv('SP500.csv')['Symbol']

    stock_df = pd.DataFrame()
    for ticker in tickers:
        stock = pd.DataFrame.from_dict([idv_stock_rec(ticker)],orient='columns')
        stock_df = pd.concat([stock_df, stock], ignore_index=True)
    return stock_df

In [102]:
stock_db = stock_rec([])

BRK.B: No data found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (period=1mo)


In [104]:
stock_db.to_csv('stock_db.csv', index=False)

In [105]:
stock_db

,key,price_history,catholic_values,esg_score,env_score,gov_score,soc_score,strongest_principle,weakest_principle,vice_products,ethical_concerns,military_involvement,health_impact
0,6,"{'Close': {0: 104.34681701660156, 1: 104.70093...",No,73.0,73.0,73.0,74.0,socialScore,governanceScore,NaN,NaN,NaN,NaN
1,10,"{'Close': {0: 79.80599975585938, 1: 80.2641906...",No,46.51,43.25,57.85,42.14,governanceScore,socialScore,NaN,NaN,NaN,NaN
2,7,"{'Close': {0: 113.93000030517578, 1: 114.01999...",No,64.0,71.0,69.0,57.0,environmentScore,socialScore,NaN,NaN,NaN,NaN
3,8,"{'Close': {0: 162.0399932861328, 1: 163.300003...",No,64.0,66.0,69.0,60.0,governanceScore,socialScore,NaN,NaN,NaN,NaN
4,1,"{'Close': {0: 353.6499938964844, 1: 359.549987...",No,72.0,77.0,78.0,66.0,governanceScore,socialScore,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,3,"{'Close': {0: 130.4600067138672, 1: 132.300003...",N/A,N/A,N/A,N/A,N/A,socialScore,socialScore,N/A,N/A,N/A,N/A
499,8,"{'Close': {0: 249.58999633789062, 1: 249.08999...",N/A,N/A,N/A,N/A,N/A,socialScore,socialScore,N/A,N/A,N/A,N/A
500,6,"{'Close': {0: 123.83000183105469, 1: 123.36000...",N/A,N/A,N/A,N/A,N/A,socialScore,socialScore,N/A,N/A,N/A,N/A
501,6,"{'Close': {0: 40.52000045776367, 1: 41.6199989...",N/A,N/A,N/A,N/A,N/A,socialScore,socialScore,N/A,N/A,N/A,N/A


In [ ]:
stock_db.to_json('stock_db.json')